In [5]:
import pandas as pd
import numpy as np
import shutil
import os
from tqdm import tqdm

from PIL import Image
import cv2
from sklearn.model_selection import train_test_split

In [2]:
def create_yolo_dataset(target_path, imgs_base_path, seq_name, df, subset):
    for frame in range(1, df.frame.max()+1):
        current_frame_df = df[df.frame == frame].reset_index(drop=True)
        img_path = imgs_base_path+'//'+'0'*(6-len(str(frame)))+f'{frame}.jpg'
        h, w, _ = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB).shape

        filename = '0'*(6-len(str(frame)))+str(frame)
        current_frame_df['x_center'] = (current_frame_df.left + current_frame_df.width/2)/w
        current_frame_df['y_center'] = (current_frame_df.top + current_frame_df.height/2)/h
        current_frame_df['width'] = current_frame_df.width/w
        current_frame_df['height'] = current_frame_df.height/w
        current_frame_df['class'] = np.zeros(len(current_frame_df), dtype=int)
        current_frame_df = current_frame_df.drop(['frame','id', 'left', 'top','-1', '-2', '-3'], axis=1)
        current_frame_df = current_frame_df[['class', 'x_center', 'y_center', 'width', 'height']]
        current_frame_df.to_csv(os.path.join(target_path.format('labels', subset), seq_name+'_'+filename+'.txt'), header=None, index=False)
        
        shutil.copy(img_path, os.path.join(target_path.format('images', subset), seq_name+'_'+filename+'.jpg'))


In [8]:
def mot2yolo(subset, seq_list):
    target_path = 'D:\datasets\MOT17\MOT17\yolo-mot17\{}\{}'
    for seq_name in tqdm(seq_list):
        base_path = f'D:\\datasets\\MOT17\\MOT17\\train\\{seq_name}'
        imgs_base_path = os.path.join(base_path, 'img1')
        gt_base_path = os.path.join(base_path, 'gt\\gt.txt')
        df = pd.read_csv(gt_base_path, names=['frame', 'id', 'left', 'top', 'width', 'height', '-1', '-2', '-3'], header=None)
        create_yolo_dataset(target_path, imgs_base_path, seq_name, df, subset)

train, test = train_test_split(os.listdir(f'D:\\datasets\\MOT17\\MOT17\\train'), test_size=0.2, random_state=42)

mot2yolo('train', train)
mot2yolo('test', test)


100%|██████████| 5/5 [01:45<00:00, 21.05s/it]
